In [37]:
import pandas as pd
import random
from datetime import datetime

## Reading the entities data

In [38]:
Entities=pd.read_excel('./Risk Profile.xlsx',sheet_name='Entities')

### since we do not have the income data I am going to remove it from the data frame also the Tampered Device? column

In [39]:
Entities.drop(['Income','Tampered Device?'],axis=1,inplace=True)

### conver the DOB column to age 

In [40]:
Entities.DOB=Entities.DOB.map(lambda e:int((pd.to_datetime('01-09-2018')-e).components[0]/365))

## Reading the classes

In [41]:
Age=pd.read_excel('./Risk Profile.xlsx',sheet_name='Age')
Occupation=pd.read_excel('./Risk Profile.xlsx',sheet_name='Occupation')
Gender=pd.read_excel('./Risk Profile.xlsx',sheet_name='Gender')
Nationalities=pd.read_excel('./Risk Profile.xlsx',sheet_name='Nationalities')
Cities=pd.read_excel('./Risk Profile.xlsx',sheet_name='Cities')

### convert the classes of Age data to range 

In [42]:
Age.Age=Age.Age.map(lambda e:range(int(e.split('-')[0]),int(e.split('-')[1])+1))

## Define a function to give each classes a random values btween [0.1,1]

In [43]:
def RiskFactor(DataFrame):
    DataFrame['RiskFactor']=pd.Series()
    for i in range(len(DataFrame)):
        DataFrame['RiskFactor'].iloc[i]=random.randint(1,10)/10

In [44]:
%%capture --no-stdout
RiskFactor(Age)
RiskFactor(Occupation)
RiskFactor(Gender)
RiskFactor(Nationalities)
RiskFactor(Cities)

## Dictionary for wieghts

In [45]:
wieght={'age':0.3,'occupation':0.1,'gender':0.2,'nationalities':0.3,'cities':0.1}

## Define a function that takes a client ID and give the risk score

In [46]:
def ClientScore(clientID):
    risk_score=0
    for c in range(len(Entities)):
            if Entities.ID.iloc[c]==clientID:
                Age_f=[Age.RiskFactor.iloc[i] for i in range(len(Age)) if Entities.DOB.iloc[c] in Age.Age[i]]
                #print(Age_f)
                Occupation_f=[Occupation.RiskFactor.iloc[i] for i in range(len(Occupation)) 
                              if Entities.Occupation.iloc[c]==Occupation.Occupation.iloc[i]]
                #print(Occupation_f)
                Gender_f=[Gender.RiskFactor.iloc[i] for i in range(len(Gender)) 
                          if Entities.Gender.iloc[c]==Gender.Gender.iloc[i]]
                #print(Gender_f)
                Nationalities_f=[Nationalities.RiskFactor.iloc[i] for i in range(len(Nationalities)) 
                                if Entities.Nationality.iloc[c].lower()==Nationalities.Nationality.iloc[i].lower()]
                #print(Nationalities_f)
                Cities_f=[Cities.RiskFactor.iloc[i] for i in range(len(Cities)) 
                         if Entities.City.iloc[c]==Cities.City.iloc[i]]
                #print(Cities_f)
                risk_score=((wieght['age']*Age_f[0])+(wieght['occupation']*Occupation_f[0])+(wieght['gender']*Gender_f[0])+(wieght['nationalities']*Nationalities_f[0])+(wieght['cities']*Cities_f[0])).round(3)
    return risk_score 

## Create a new column for risk score 

In [48]:
Entities['RiskScore']=pd.Series()
for i in range(len(Entities)):
    Entities['RiskScore'][i]=ClientScore(Entities.ID[i])

/Users/amjadalabdullah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Save the file

In [51]:
Entities.to_csv('./Entities')